In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import dlib
import os, pickle
import re
import signal
import operator
from sklearn import svm
from sklearn import tree

## Rearrange Embeddings for training

Rearrange data to be compatible for training iwth sklearn

In [ ]:
def rearrange_people_reps(people_reps):
    all_reps = []
    classes = []
    for person in range(len(people_reps)):
        for rep in range(len(people_reps[person])):
            all_reps.append(people_reps[person][rep])
            classes.append(person)
    return all_reps, classes            

## Load embeddings

Load all Representation of dataset (LFW)

In [ ]:
names = np.load('models/facenet_names.npy')
people_reps = np.load('models/facenet_reps.npy')

Train and classify with SVM

In [ ]:
reps, classes = rearrange_people_reps(people_reps)

In [ ]:
lin_clf = svm.SVC(probability=True)

In [ ]:
lin_clf.fit(reps, classes) 

In [ ]:
probabilities = lin_clf.predict_proba(rep.reshape(1,-1))
output = np.argmax(probabilities[0])
print(probabilities[0])


In [ ]:
classification = names[output]
probability = probabilities[0][output]

In [ ]:
print(classification)
print('Probability: ' + str(probability))

Decision Tree

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(reps, classes)

In [ ]:
probabilities = clf.predict_proba(rep.reshape(1,-1))
output = np.argmax(probabilities[0])

In [ ]:
classification = names[output]
probability = probabilities[0][output]

In [ ]:
print(classification)
print('Probability: ' + str(probability))

In [ ]:
pickle.dump( clf, open( "models/tree.b", "wb" ) )

In [ ]:
t = pickle.load( open( "models/tree.b", "rb" ) )

Own Model

In [ ]:
names = np.load('models/own_names.npy')
people_reps = np.load('models/own_reps.npy')

In [ ]:
k = 3
neighbours = getNeighbours(people_reps, rep, k)

In [ ]:
output = getResponse(neighbours)

In [ ]:
classification = names[1]
distance = neighbours[0][2]

In [ ]:
print(classification)
print('Distance: ' + str(distance))

In [ ]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [ ]:
X, classes = rearrange_people_reps(people_reps)
X = np.array(X)
classes = np.array(classes)

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=2, solver='eigen', shrinkage='auto')
people_lda = lda.fit(X, classes).transform(X)
print(X.shape, classes.shape, people_lda.shape)

In [ ]:
import matplotlib.cm as cm

plt.figure(dpi=96) # Make the chart bigger
plt.xticks([])
plt.yticks([])

# A set of points for each person gets its own color
colors = cm.rainbow(np.linspace(0, 1, len(names)))
plots = []
prev = 0
for p, color in zip(people_reps, colors):
    s = np.array(int(p.shape[0]/2))
    print(s)
    person_lda = people_lda[prev:prev+p.shape[0]]
    print(person_lda)
    prev = p.shape[0]
    # Split the (x, y) array into two arrays: x and y
    x, y = tuple(np.split(person_lda, 2, axis=1))
    plots.append(plt.scatter(x, y, color=color))
    plt.legend(plots, names, loc='best', shadow=False, scatterpoints=1)